# Basic RNA circuit simulation 



In [1]:

from bioreaction.simulation.med_sim import get_int_med_model, basic_de, MedSimParams, MedSimInternalState
from bioreaction.model.data_tools import construct_model_fromnames
from bioreaction.simulation.basic_sim import basic_de_sim, convert_model, BasicSimParams, BasicSimState
from bioreaction.model.data_containers import Impulse
import bioreaction

from tqdm import tqdm
import diffrax as dfx
import jax
import pandas as pd
import jax.random as jr
import jax.numpy as jnp
import numpy as np
from copy import deepcopy
from functools import partial
import os
import sys

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-paper')
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


from src.srv.io.manage.script_manager import script_preamble
from src.utils.results.analytics.timeseries import generate_analytics, get_precision, get_sensitivity, get_step_response_times
from src.utils.results.analytics.naming import get_true_names_analytics, get_true_interaction_cols
from src.utils.misc.units import per_mol_to_per_molecule
from src.utils.data.data_format_tools.common import load_json_as_dict
from src.utils.common.setup_new import prepare_config
from src.utils.circuit.agnostic_circuits.circuit_manager_new import CircuitModeller
from tests_local.shared import five_circuits

config = load_json_as_dict('../tests_local/configs/simple_circuit.json')


In [2]:



def update_model_rates(model, a=None, d=None, ka=None, kd=None):
    for i, r in enumerate(model.reactions):
        if not r.input:  # 0 -> RNA
            if a is not None:
                model.reactions[i].forward_rate = a[model.species.index(
                    r.output[0])]
                model.reactions[i].reverse_rate = 0
        elif not r.output:  # RNA -> 0
            if d is not None:
                model.reactions[i].forward_rate = d[model.species.index(
                    r.input[0])]
                model.reactions[i].reverse_rate = 0
        else: # unbound RNA -> bound RNA
            if ka is not None:
                model.reactions[i].forward_rate = ka[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
            if kd is not None:
                model.reactions[i].reverse_rate = kd[model.species.index(r.input[0]),
                                                     model.species.index(r.input[1])]
    return model


def make_params(model, scale_rates=True):
    sim_model = convert_model(model)

    if scale_rates:
        m = np.max([sim_model.forward_rates.max(), sim_model.reverse_rates.max()])
    else:
        m = 1
    inputs = sim_model.inputs
    outputs = sim_model.outputs
    forward_rates = sim_model.forward_rates/m
    reverse_rates = sim_model.reverse_rates/m
    return inputs, outputs, forward_rates, reverse_rates, m

Make a model :)

In [3]:
config = prepare_config(config)
circuits, config, result_writer = five_circuits(config)


xla_bridge.py:backends():355: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:  INFO
xla_bridge.py:backends():355: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Host CUDA Interpreter INFO
xla_bridge.py:backends():355: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client' INFO
xla_bridge.py:backends():355: Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this. INFO
experiment_loading.py:load_units():35: Units unknown - supply a valid experiment directory instead of ../tests_local/configs/binding_rates_dissociation/0_weak_binding_rates_dissociation.csv WARNING
experiment_loading.py:load_units():35: Units unknown - supply a valid experi

In [4]:

circuit_modeller = CircuitModeller(result_writer=result_writer, config=config)
circuit_modeller.batch_circuits(
    circuits=circuits,
    methods={
        "compute_interactions": {},
        "init_circuits": {'batch': True},
        'simulate_signal_batch': {'ref_circuit': None,
                                  'batch': config['simulation']['use_batch_mutations']},
        'write_results': {'no_visualisations': False, # config['experiment']['no_visualisations'],
                          'no_numerical': False} #config['experiment']['no_numerical']}
    }
)


simulator.py:check_IntaRNA_path():116: Could not detect IntaRNA on system: CompletedProcess(args='which IntaRNA', returncode=1, stdout='', stderr='') WARNING
circuit_manager_new.py:prepare_internal_funcs():446: Signal differs between circuits, but only first signal used for simulation. WARNING
circuit_manager_new.py:batch_circuits():516: 	From 5 circuits, a total of 5 mutated circuits will be simulated. WARNING
circuit_manager_new.py:batch_circuits():523: 		Starting new round of viable circuits (0 - 5 / 5) WARNING
circuit_manager_new.py:batch_circuits():541: 		Making subcircuits 0 - 5 took 1.3e-05s WARNING
circuit_manager_new.py:batch_circuits():547: 	Batching 0 - 5 circuits (out of 0 - 5 (total: 5)) (Circuits: 0 - 5 of 5) WARNING
circuit_manager_new.py:run_batch():578: 		Running 5 Subcircuits - 0_weak: compute_interactions WARNING
circuit_manager_new.py:run_batch():614: 		Method compute_interactions took 0.063911s WARNING
circuit_manager_new.py:run_batch():578: 		Running 5 Subcircuits

Steady states:  100  iterations.  45  left to steady out.  0:00:02.988319


KeyboardInterrupt: 